In [1]:
import numpy as np
import pandas as pd

import librosa
import os

In [2]:
starting_directory = os.getcwd()
archive = 'a:\\UE Studies\\Machine-Learning\\Project\\noise_can\\Work\\archive'

if not (os.path.isdir("archive_dataframes")):
    os.mkdir("archive_dataframes")
    print("archive_dataframes file or directory has been created")
else: print("directory exist")

directory exist


In [5]:
df_UrbanSound8K = pd.read_csv("UrbanSound8K.csv")
df_UrbanSound8K.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [6]:
# crating dataframe for each path
def create_dataframe(pt,classs):
    # Loadinng the audio file:
    x, sr = librosa.load(pt)

    # Increasing the length of the length of audio file to 4 sec:
    # To make add dataframes of the same shape or size:
    while True:
        if len(x) < (sr*4+1):
            x = np.concatenate((x,[0]*sr,x), axis=0)
        else:
            x = x[:sr*4+1]
            break
    mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc= 40)                                # Feature Extraction:
    mfccs_features = np.mean(mfccs.T,axis=0)                                            # Taking average for each feature
    df = pd.DataFrame(np.hstack((np.array(pt[:-4]), mfccs_features, np.array(classs)))) # Making Data Frames with these deature
    
    return df


In [7]:
# This piece of code is just to save the dataframe in to csv file in the local machine.

def make_dataframe(df,folder):
    os.chdir("archive_dataframes")
    if os.path.exists(folder + "_df_file.csv"):
        os.remove(folder + "_df_file.csv")
    
    print(df.shape, "     ", end = " ")
    df.to_csv(folder + "_df_file.csv" , index=False)

In [8]:
# this code will return row nuber in dataframe Urbansound8K corresponds to name of file given without .wav in it

def get_row_number(x):
    for i in range(len(df_UrbanSound8K.slice_file_name)):
        if df_UrbanSound8K.slice_file_name[i] == x:
            break
    return i

In [9]:
os.chdir(archive)                                               # archive folder contains 10 folders of each aroud 800 audio files
list_of_folders = os.listdir()

for folder in list_of_folders:                                  # Iterating over archive folder
    dataframe = pd.DataFrame({})
    if "fold" in folder:
        os.chdir(archive + "\\" + folder)

        for x in os.listdir():                                  # Changing directory in to one of folders inside archive folder
            if ".wav" in x:      
                df = create_dataframe(x,df_UrbanSound8K.iloc[get_row_number(x), 7]) # Calling out the function we created earlier
                dataframe = pd.concat([dataframe, df], axis=1)  # Conbining all returned dataframes into one dataframe

        dataframe = dataframe.T
        make_dataframe(dataframe,folder)                        # Crating csv file by calling a function we crated earlier.
        print(folder + " saving was done")

os.chdir(starting_directory)                                    # Changing directory back to original directory

(873, 42)       fold1 saving was done
(837, 42)       fold10 saving was done
(888, 42)       fold2 saving was done
(925, 42)       fold3 saving was done
(990, 42)       fold4 saving was done
(936, 42)       fold5 saving was done
(823, 42)       fold6 saving was done
(838, 42)       fold7 saving was done
(806, 42)       fold8 saving was done
(816, 42)       fold9 saving was done


In [10]:
a = pd.read_csv("fold10_df_file.csv")                   # Checking one of the csv files
a.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,100648-1-0-0,-458.852208,54.472987,-0.354782,-5.420509,-4.833549,-8.203389,-15.698002,-3.326693,-4.583599,...,-0.482120,-0.922345,1.762407,-3.423761,-2.290337,-1.124767,1.790613,-2.936575,-0.412744,car_horn
1,100648-1-1-0,-448.263515,54.760387,-8.887156,1.159189,-8.107995,-12.153162,-7.872774,-3.384088,-9.733895,...,-1.779253,-1.320522,2.485539,-4.898012,-2.803906,3.534088,1.255909,-7.499753,2.112319,car_horn
2,100648-1-2-0,-550.542407,27.620024,-5.018342,5.888297,-5.072524,-6.054349,-2.783565,-3.432212,-7.160883,...,0.578030,-2.039364,-0.300180,-0.804519,-1.299412,-2.413588,0.574557,-2.323122,-0.994150,car_horn
3,100648-1-3-0,-342.377805,68.933450,-13.504516,11.507217,-7.056891,-11.519361,-8.962671,-1.837157,-19.751329,...,2.881917,-1.053040,-1.786281,-3.925069,2.372586,-3.171969,-4.676545,-1.032859,2.522283,car_horn
4,100648-1-4-0,-209.303421,46.283464,-6.854885,6.537487,0.573484,-12.509450,-15.645269,3.419694,-17.512218,...,0.936365,-0.461889,-0.828304,-3.677241,2.747718,-1.457215,-2.387974,-2.699965,1.337122,car_horn
